# 論拠データを整理する
論拠データを作成するためのタスクです。<br>
リサーチフローにおいて論拠データは公開する論文の根拠となるデータとして作成します。<br>
このタスクでは論拠データが公開に適したものになるよう、実験データから論文に必要のないものを除外して整理する作業をお行ってください。<br>
また、論拠データは論文に即したものになるよう、論文の内容に合わせて適宜修正してください。

## 論拠データを整理する
以下の手順に従って論拠データの整理を行ってください。


### 1. リサーチフローのデータをコピーする
親サブフローになっている実験サブフローから必要な実験データをコピーして下さい。<br>
「リサーチフローのデータをコピーする」セルを実行することで親サブフローになっている実験サブフローのフォルダと論拠データを格納するためのフォルダを表示することができます。<br>
以下の図の説明に従ってデータのコピーを行ってください。<br>
<br>
![ダウンロード手順](./images/RF004003_download.png)<br>
<br>
![アップロード手順](./images/RF004003_upload.png)<br>
<br>
親サブフローを変更する場合はメインメニューに戻り親子関係を変更した後、このタスクを実行してください。<br>
また、リサーチフローに保存していないデータを使用する場合はこのタスクで論拠データを格納するためのフォルダに追加してください。
<br>
※研究データのほか、ガバナンスシートなどを公開する場合はセル実行時に表示されるボタンを押下してください。


In [ ]:
# リサーチフローのデータをコピーする
import os

from IPython.core.display import Javascript
import panel as pn

from library.task_director import TaskDirector
from library.utils.access import open_data_folder
from library.utils.config import path_config, message as msg_config
from library.utils.error import NotFoundSubflowDataError
from library.utils.setting.research_flow_status import get_subflow_type_and_id, ResearchFlowStatusOperater
from library.utils.widgets import MessageBox


notebook_name = 'organize_argument_data.ipynb'

class AccessSubflowFolder(TaskDirector):
    """親となる実験サブフローと実行中のサブフローの論拠データのフォルダを表示するクラスです。

    Attributes:
        instance:
            working_path(str): 実行Notebookファイルパス
            _msg_output(MessageBox): メッセージ出力用のボックス
            _form_section(pn.WidgetBox): ボタン等の出力を格納するためのボックス

    """
    def __init__(self, working_path: str):
        """Displyクラスのコンストラクタです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

        # メッセージボックスの設定
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        # 出力用フォームの設定
        self._form_section = pn.WidgetBox()

    @TaskDirector.task_cell("1")
    def generate_folder_buttons(self):
        """データフォルダを表示するためのボタンを生成するメソッドです。"""

        self.doing_task()

        _, subflow_id = get_subflow_type_and_id(self.working_path)
        parents_list = self.get_parent_info(subflow_id)

        # 表示する実験サブフローが存在しない場合はメッセージを表示する
        if self._msg_output.has_message():
            self._form_section.append(self._msg_output)

        else:
            # 取得した実験サブフローのフォルダを開くボタンの作成
            old_part= os.path.join("writing", subflow_id)
            for parent_name, parent_id in parents_list.items():
                new_part = os.path.join("experiment", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)

                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config.get('organize_argument_data', 'title_experiment')}{parent_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

            # 論拠データフォルダを開くボタンの作成
            pn.extension(raw_css=[".custom-card { background-color:rgba(255, 255, 0, 0.1);}"])
            button = open_data_folder(self.working_path, "argument_data")
            card = pn.Card(
                button,
                title = f"{msg_config.get('organize_argument_data', 'title_argument_data')}",
                width = 550,
                height = 100,
                css_classes = ["custom-card"]
            )
            self._form_section.append(card)

        self.done_task()
        # 表示する
        pn.extension()
        display(self._form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_parent_info(self, current_subflow: str) -> dict:
        """親サブフローとなっている実験サブフローを取得するメソッドです。

        Args:
            current_subflow (str): 現在実行中のサブフローのID

        Returns:
            dict: 親サブフローとなっている実験サブフローの情報

        """
        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(abs_root)
        reserch_flow_status_operater = ResearchFlowStatusOperater(research_flow_status_file_path)

        # 全ての親サブフローIDを取得
        parent_ids = reserch_flow_status_operater.get_parent_ids(3, current_subflow) # 3:論文執筆フェーズのシーケンス番号

        # 実験サブフローのみを取得
        parent_subflow = {}
        for parent_id in parent_ids:
            try:
                subflow_name = reserch_flow_status_operater.get_flow_name(2, parent_id) # 2:実験実験フェーズのシーケンス番号
                parent_subflow[subflow_name] = parent_id
            except NotFoundSubflowDataError:
                continue

        # 親サブフローに実験サブフローが存在しない場合はメッセージを格納
        if not parent_subflow:
            self._msg_output.update_warning(msg_config.get("organize_argument_data", "not_found_experimet_subflow"))

        return parent_subflow

AccessSubflowFolder(os.path.abspath('__file__')).generate_folder_buttons()

### 2. コピーしたデータを整理する
コピーしたデータのフォルダを表示します。<br>
フォルダ構成や名前の編集、不要なデータの削除などを行い、論拠データを論文に即したものになるよう整理してください。

In [ ]:
# コピーしたデータを表示する
import os

import panel as pn
from IPython.core.display import Javascript
from IPython.display import display

from library.utils.access import open_data_folder


folder_name = 'argument_data'
button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

## 論拠データにメタデータを付与する
論拠データのメタデータ登録を行ってください。<br>
セルを実行してメタデータの登録を行った後、このタスクに戻り作業を再開してください。<br>
GakuNin RDMにメタデータを登録する場合はGakuNin RDM上で直接行ってください。<br>

In [ ]:
# メタデータを登録するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_register_metadata():
    """検証するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/metadata.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_register_metadata_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_register_metadata()

## 再現性を確認する
メタデータの検証と再現性の確認を行ってください。<br>
<br>
※再現性を確認する機能は現在開発中です。


### メタデータを検証する
論拠データのメタデータの検証を行ってください。<br>
セルを実行してメタデータの検証を行った後、このタスクに戻り作業を再開してください。

In [ ]:
# 検証するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_validate():
    """検証するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/validate.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_validate_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_validate()

## 収集した文献を整理する
論文執筆に向けて収集した文献を整理し、リサーチフローにアップロードしてください。<br>
また、参考文献は論文に即したものになるよう、論文の内容に合わせて適宜編集してください。

In [ ]:
# 文献データフォルダを表示する
import os

from IPython.core.display import Javascript
from IPython.display import display
import panel as pn

from library.utils.access import open_data_folder


folder_name = 'references'
button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

##  GakuNin RDMに保存する
タスクの状態をGakuNin RDMに保存します。

In [ ]:
# GakuNin RDMに保存する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import path_config
from library.task_director import TaskDirector
from library.utils.setting import get_data_dir

script_file_name = 'organize_argument_data'
notebook_name = script_file_name+'.ipynb'


class DataSaver(TaskDirector):
    """GRDMに保存するクラスです。

    Attributes:
        instance:
            _abs_root_path (str): 絶対パス
            save_form_box(pn.WidgetBox):フォームを格納する。
            save_msg_output(Message):ユーザーに提示するメッセージを格納する。
    """

    def __init__(self, working_path: str) -> None:
        """DataSaver コンストラクタメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

    @TaskDirector.task_cell("1")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""

        # フォーム定義
        data_dir = get_data_dir(self.working_path)
        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(abs_root)
        source = [os.path.join(data_dir, 'argument_data'), os.path.join(data_dir, 'references'), research_flow_status_file_path]
        self.define_save_form(source)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

DataSaver(working_path=os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os

from library.task_director import TaskDirector

script_file_name = "organize_argument_data"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()